<a href="https://colab.research.google.com/github/ferdcoz/mcd-ast/blob/original/TP2/Codigo/TP2_Skforecast_v1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install skforecast session_info

In [ ]:
!python --version

Python 3.10.12


## Librerias

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

## Funciones

In [ ]:
def plot_data_split(train_data, val_data, test_data, column):
  fig = go.Figure()

  trace1 = go.Scatter(x=train_data.index, y=train_data[column], name="Training", mode="lines", line=dict(color="#440154"))
  trace2 = go.Scatter(x=val_data.index, y=val_data[column], name="Validation", mode="lines", line=dict(color="#3b528b"))
  trace3 = go.Scatter(x=test_data.index, y=test_data[column], name="Testing", mode="lines", line=dict(color="#21918c"))

  fig.add_trace(trace1)
  fig.add_trace(trace2)
  fig.add_trace(trace3)

  fig.update_layout(
      title=f"{column} - Data Split",
      xaxis_title="Fecha",
      yaxis_title="Cantidad",
      width  = 800,
      height = 400,
      margin=dict(l=20, r=20, t=35, b=20),
      legend=dict(
          orientation="h",
          yanchor="top",
          y=1.1,
          xanchor="left",
          x=0.001
      )
  )
  fig.show()

In [ ]:
def plot_prediction(train_data, val_data, test_data, predictions, column):
  fig = go.Figure()

  trace1 = go.Scatter(x=train_data.index, y=train_data[column], name="Training", mode="lines", line=dict(color="#440154"))
  trace2 = go.Scatter(x=val_data.index, y=val_data[column], name="Validation", mode="lines", line=dict(color="#3b528b"))
  trace3 = go.Scatter(x=test_data.index, y=test_data[column], name="Testing", mode="lines", line=dict(color="#21918c"))
  trace4 = go.Scatter(x=predictions.index, y=predictions['pred'], name="Prediction", mode="lines", line=dict(color="#fde725"))

  fig.add_trace(trace1)
  fig.add_trace(trace2)
  fig.add_trace(trace3)
  fig.add_trace(trace4)

  fig.update_layout(
      title=f"{column} predictions",
      xaxis_title="Fecha",
      yaxis_title="Cantidad",
      width  = 800,
      height = 400,
      margin=dict(l=20, r=20, t=35, b=20),
      legend=dict(
          orientation="h",
          yanchor="top",
          y=1.1,
          xanchor="left",
          x=0.001
      )
  )
  fig.show()

## Data

In [ ]:
# Incluyendo CC y la data como viene
# url = 'https://raw.githubusercontent.com/ferdcoz/mcd-ast/original/TP2/Dataset/ST_ATENCION_ORIGNAL.csv'

# Excluyendo CC y sin 0s
url = 'https://raw.githubusercontent.com/ferdcoz/mcd-ast/original/TP2/Dataset/ST_ATENCION.csv'
data = pd.read_csv(url, delimiter=';')
data.head()

,FECHA,WAPP,TP,TAD,PP
0,2022-03-15,19,78,53,9
1,2022-03-16,12,70,26,12
2,2022-03-17,14,66,41,14
3,2022-03-18,67,50,20,9
4,2022-03-21,45,65,44,8


In [ ]:
data['FECHA'] = pd.to_datetime(data['FECHA'], format='%Y-%m-%d')
data = data.set_index('FECHA')
data = data.asfreq('B')
data = data.sort_index()
data.head()

,WAPP,TP,TAD,PP
FECHA,,,,
2022-03-15,19.0,78.0,53.0,9.0
2022-03-16,12.0,70.0,26.0,12.0
2022-03-17,14.0,66.0,41.0,14.0
2022-03-18,67.0,50.0,20.0,9.0
2022-03-21,45.0,65.0,44.0,8.0


In [ ]:
data = data.fillna(data.median())

In [ ]:
#data_CC = data[['CC']]
data_WAPP = data[['WAPP']]
data_TP = data[['TP']]
data_TAD = data[['TAD']]
data_PP = data[['PP']]

## Modelos

### XGBoost

In [ ]:
xgb_forecaster = ForecasterAutoreg(
                 regressor = XGBRegressor(random_state=123),
                 lags = 4
             )

### LightGBM

In [ ]:
lgbm_forecaster = ForecasterAutoreg(
                 regressor = LGBMRegressor(random_state=123),
                 lags = 4
             )

### Metricas

In [ ]:
metrics = pd.DataFrame(columns=['Variable', 'XGBoost MSE', 'LightGBM MSE'])

## CC

In [ ]:
# data = data_CC

### 1. Split data

In [ ]:
# Split de 80/10/10
# total_rows = len(data)
# train_rows = int(0.8 * total_rows)
# val_rows = int(0.1 * total_rows)

# train_data = data.iloc[:train_rows]
# val_data   = data.iloc[train_rows:train_rows + val_rows]
# test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
# plot_data_split(train_data, val_data, test_data, "CC")

### 3. Hiperparametros

In [ ]:
# train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
# param_grid_xgb = {
#     'n_estimators': [100, 500],
#     'max_depth': [3, 5, 10],
#     'num_leaves' : [8, 32, 1024],
#     'learning_rate': [0.01, 0.1]
# }

# # Lags used as predictors
# lags_grid = [4, 24]

# results_grid_xgb = grid_search_forecaster(
#                    forecaster         = xgb_forecaster,
#                    y                  = train_val, # Train and validation data
#                    param_grid         = param_grid_xgb,
#                    lags_grid          = lags_grid,
#                    steps              = 15,
#                    refit              = False,
#                    metric             = 'mean_squared_error',
#                    initial_train_size = len(train_data), # Model is trained with training data
#                    fixed_train_size   = False,
#                    return_best        = True,
#                    verbose            = False
#                )

In [ ]:
# param_grid_lgbm = {
#     'n_estimators': [100, 500],
#     'max_depth': [3, 5, 10],
#     'num_leaves' : [8, 32, 1024],
#     'learning_rate': [0.01, 0.1]
# }

# results_grid_lgbm = grid_search_forecaster(
#                    forecaster         = lgbm_forecaster,
#                    y                  = train_val, # Train and validation data
#                    param_grid         = param_grid_lgbm,
#                    lags_grid          = lags_grid,
#                    steps              = 15,
#                    refit              = False,
#                    metric             = 'mean_squared_error',
#                    initial_train_size = len(train_data), # Model is trained with training data
#                    fixed_train_size   = False,
#                    return_best        = True,
#                    verbose            = False
#                )

In [ ]:
# results_grid_xgb.head()

In [ ]:
# results_grid_lgbm.head()

### 4. Prediccion

In [ ]:
# metric_xgb, predictions_xgb = backtesting_forecaster(
#                             forecaster         = xgb_forecaster,
#                             y                  = data.squeeze(),
#                             initial_train_size = len(train_data),
#                             fixed_train_size   = False,
#                             steps              = 15,
#                             refit              = False,
#                             metric             = 'mean_squared_error',
#                             verbose            = False # Change to True to see detailed information
#                       )

# print(f"Backtest error: {metric_xgb:.2f}")
# predictions_xgb.head()

In [ ]:
# metric_lgbm, predictions_lgbm = backtesting_forecaster(
#                             forecaster         = lgbm_forecaster,
#                             y                  = data.squeeze(),
#                             initial_train_size = len(train_data),
#                             fixed_train_size   = False,
#                             steps              = 15,
#                             refit              = False,
#                             metric             = 'mean_squared_error',
#                             verbose            = False # Change to True to see detailed information
#                       )

# print(f"Backtest error: {metric_lgbm:.2f}")
# predictions_lgbm.head()

### 5. Plot prediccion

In [ ]:
# plot_prediction(train_data, val_data, test_data, predictions_xgb, "CC")

In [ ]:
# plot_prediction(train_data, val_data, test_data, predictions_lgbm, "CC")

## WAPP

In [ ]:
data = data_WAPP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "WAPP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid_xgb = grid_search_forecaster(
                   forecaster         = xgb_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_xgb,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:14:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: {

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 500, 'num_leaves': 8}
  Backtesting metric: 99.74881872622291



In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

results_grid_lgbm = grid_search_forecaster(
                   forecaster         = lgbm_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_lgbm,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
results_grid_xgb.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
45,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",99.748819,0.01,5.0,500.0,8.0
46,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",99.748819,0.01,5.0,500.0,32.0
47,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",99.748819,0.01,5.0,500.0,1024.0
59,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",102.991123,0.10,3.0,500.0,1024.0
57,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",102.991123,0.10,3.0,500.0,8.0


In [ ]:
results_grid_lgbm.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
53,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",103.531558,0.01,10.0,500.0,1024.0
52,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",103.531558,0.01,10.0,500.0,32.0
66,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",108.512958,0.10,10.0,100.0,8.0
9,"[1, 2, 3, 4]","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",111.480046,0.01,5.0,500.0,8.0
11,"[1, 2, 3, 4]","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",112.035054,0.01,5.0,500.0,1024.0


### 4. Prediccion

In [ ]:
metric_xgb, predictions_xgb = backtesting_forecaster(
                            forecaster         = xgb_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_xgb:.2f}")
predictions_xgb.head()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 79.09


,pred
2023-06-09,29.707302
2023-06-12,30.393841
2023-06-13,39.193989
2023-06-14,46.290821
2023-06-15,32.549614


In [ ]:
metric_lgbm, predictions_lgbm = backtesting_forecaster(
                            forecaster         = lgbm_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_lgbm:.2f}")
predictions_lgbm.head()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000095 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1008
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 24
[LightGBM] [Info] Start training from score 30.913043
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 73.53


,pred
2023-06-09,30.086906
2023-06-12,28.876264
2023-06-13,30.209988
2023-06-14,33.214044
2023-06-15,28.853494


In [ ]:
metrics = metrics.append({'Variable': 'WAPP', 'XGBoost MSE': metric_xgb, 'LightGBM MSE': metric_lgbm}, ignore_index=True)

<ipython-input-189-de3228231bfe>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_xgb, "WAPP")

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_lgbm, "WAPP")

## TP

In [ ]:
data = data_TP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "TP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid_xgb = grid_search_forecaster(
                   forecaster         = xgb_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_xgb,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:42] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:15:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: {

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500, 'num_leaves': 8}
  Backtesting metric: 971.5340119251923



In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

results_grid_lgbm = grid_search_forecaster(
                   forecaster         = lgbm_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_lgbm,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
results_grid_xgb.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
57,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",971.534012,0.1,3.0,500.0,8.0
59,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",971.534012,0.1,3.0,500.0,1024.0
58,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",971.534012,0.1,3.0,500.0,32.0
65,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",1092.979351,0.1,5.0,500.0,1024.0
63,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",1092.979351,0.1,5.0,500.0,8.0


In [ ]:
results_grid_lgbm.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
63,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",760.349444,0.1,5.0,500.0,8.0
59,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",789.143099,0.1,3.0,500.0,1024.0
58,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",789.143099,0.1,3.0,500.0,32.0
57,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",789.143099,0.1,3.0,500.0,8.0
71,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",804.005765,0.1,10.0,500.0,1024.0


### 4. Prediccion

In [ ]:
metric_xgb, predictions_xgb = backtesting_forecaster(
                            forecaster         = xgb_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_xgb:.2f}")
predictions_xgb.head()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:51] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 1468.96


,pred
2023-06-09,156.035095
2023-06-12,173.570129
2023-06-13,171.539185
2023-06-14,144.258972
2023-06-15,138.950485


In [ ]:
metric_lgbm, predictions_lgbm = backtesting_forecaster(
                            forecaster         = lgbm_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_lgbm:.2f}")
predictions_lgbm.head()

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000050 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1720
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 24
[LightGBM] [Info] Start training from score 171.785953
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 1320.91


,pred
2023-06-09,165.606318
2023-06-12,144.187646
2023-06-13,157.403670
2023-06-14,151.234221
2023-06-15,146.911088


In [ ]:
metrics = metrics.append({'Variable': 'TP', 'XGBoost MSE': metric_xgb, 'LightGBM MSE': metric_lgbm}, ignore_index=True)

<ipython-input-202-ada880f821bc>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_xgb, "TP")

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_lgbm, "TP")

## TAD

In [ ]:
data = data_TAD

### 1. Split Data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "TAD")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid_xgb = grid_search_forecaster(
                   forecaster         = xgb_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_xgb,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:52] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:54] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:16:55] WARNING: /workspace/src/learner.cc:742: 
Parameters: {

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24] 
  Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'num_leaves': 1024}
  Backtesting metric: 71.48580729752867



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:09] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

results_grid_lgbm = grid_search_forecaster(
                   forecaster         = lgbm_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_lgbm,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
results_grid_xgb.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
38,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",71.485807,0.01,3.0,100.0,1024.0
37,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",71.485807,0.01,3.0,100.0,32.0
36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",71.485807,0.01,3.0,100.0,8.0
54,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",72.591678,0.10,3.0,100.0,8.0
56,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",72.591678,0.10,3.0,100.0,1024.0


In [ ]:
results_grid_lgbm.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
36,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",70.630644,0.01,3.0,100.0,8.0
38,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",70.630644,0.01,3.0,100.0,1024.0
37,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 3, 'n_est...",70.630644,0.01,3.0,100.0,32.0
42,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",72.730586,0.01,5.0,100.0,8.0
43,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",73.201277,0.01,5.0,100.0,32.0


### 4. Prediccion

In [ ]:
metric_xgb, predictions_xgb = backtesting_forecaster(
                            forecaster         = xgb_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_xgb:.2f}")
predictions_xgb.head()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 54.18


,pred
2023-06-09,34.264656
2023-06-12,35.283241
2023-06-13,35.543991
2023-06-14,35.982845
2023-06-15,36.343983


In [ ]:
metric_lgbm, predictions_lgbm = backtesting_forecaster(
                            forecaster         = lgbm_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_lgbm:.2f}")
predictions_lgbm.head()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000102 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 992
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 24
[LightGBM] [Info] Start training from score 35.628763
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 54.03


,pred
2023-06-09,33.418196
2023-06-12,36.028280
2023-06-13,36.638260
2023-06-14,35.907138
2023-06-15,36.334821


In [ ]:
metrics = metrics.append({'Variable': 'TAD', 'XGBoost MSE': metric_xgb, 'LightGBM MSE': metric_lgbm}, ignore_index=True)

<ipython-input-215-5298a3cbcb43>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_xgb, "TAD")

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_lgbm, "TAD")

## PP

In [ ]:
data = data_PP

### 1. Split data

In [ ]:
# Split de 80/10/10
total_rows = len(data)
train_rows = int(0.8 * total_rows)
val_rows = int(0.1 * total_rows)

train_data = data.iloc[:train_rows]
val_data   = data.iloc[train_rows:train_rows + val_rows]
test_data  = data.iloc[train_rows + val_rows:]

### 2. Plot split

In [ ]:
plot_data_split(train_data, val_data, test_data, "PP")

### 3. Hiperparametros

In [ ]:
train_val = data.iloc[:train_rows + val_rows].squeeze()

In [ ]:
param_grid_xgb = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

# Lags used as predictors
lags_grid = [4, 24]

results_grid_xgb = grid_search_forecaster(
                   forecaster         = xgb_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_xgb,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:44] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:18:45] WARNING: /workspace/src/learner.cc:742: 
Parameters: {

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [1 2 3 4] 
  Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100, 'num_leaves': 1024}
  Backtesting metric: 39.14041526608046



/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:19:18] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




In [ ]:
param_grid_lgbm = {
    'n_estimators': [100, 500],
    'max_depth': [3, 5, 10],
    'num_leaves' : [8, 32, 1024],
    'learning_rate': [0.01, 0.1]
}

results_grid_lgbm = grid_search_forecaster(
                   forecaster         = lgbm_forecaster,
                   y                  = train_val, # Train and validation data
                   param_grid         = param_grid_lgbm,
                   lags_grid          = lags_grid,
                   steps              = 15,
                   refit              = False,
                   metric             = 'mean_squared_error',
                   initial_train_size = len(train_data), # Model is trained with training data
                   fixed_train_size   = False,
                   return_best        = True,
                   verbose            = False
               )

Number of models compared: 72.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

params grid:   0%|          | 0/36 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
results_grid_xgb.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
26,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",39.140415,0.1,5.0,100.0,1024.0
24,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",39.140415,0.1,5.0,100.0,8.0
25,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",39.140415,0.1,5.0,100.0,32.0
54,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",40.642179,0.1,3.0,100.0,8.0
55,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",40.642179,0.1,3.0,100.0,32.0


In [ ]:
results_grid_lgbm.head()

,lags,params,mean_squared_error,learning_rate,max_depth,n_estimators,num_leaves
61,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",38.957586,0.10,5.0,100.0,32.0
62,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",38.957586,0.10,5.0,100.0,1024.0
46,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",39.188581,0.01,5.0,500.0,32.0
47,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{'learning_rate': 0.01, 'max_depth': 5, 'n_est...",39.188581,0.01,5.0,500.0,1024.0
24,"[1, 2, 3, 4]","{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",40.261730,0.10,5.0,100.0,8.0


### 4. Prediccion

In [ ]:
metric_xgb, predictions_xgb = backtesting_forecaster(
                            forecaster         = xgb_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_xgb:.2f}")
predictions_xgb.head()

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning:

[23:19:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.




  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 36.87


,pred
2023-06-09,15.868246
2023-06-12,13.895844
2023-06-13,13.726269
2023-06-14,13.621121
2023-06-15,15.023261


In [ ]:
metric_lgbm, predictions_lgbm = backtesting_forecaster(
                            forecaster         = lgbm_forecaster,
                            y                  = data.squeeze(),
                            initial_train_size = len(train_data),
                            fixed_train_size   = False,
                            steps              = 15,
                            refit              = False,
                            metric             = 'mean_squared_error',
                            verbose            = False # Change to True to see detailed information
                      )

print(f"Backtest error: {metric_lgbm:.2f}")
predictions_lgbm.head()

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000164 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1056
[LightGBM] [Info] Number of data points in the train set: 299, number of used features: 24
[LightGBM] [Info] Start training from score 22.752508
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, 

  0%|          | 0/6 [00:00<?, ?it/s]

Backtest error: 31.71


,pred
2023-06-09,16.364069
2023-06-12,18.534745
2023-06-13,18.226381
2023-06-14,15.873389
2023-06-15,11.715580


In [ ]:
metrics = metrics.append({'Variable': 'PP', 'XGBoost MSE': metric_xgb, 'LightGBM MSE': metric_lgbm}, ignore_index=True)

<ipython-input-228-dc06cf4e6f76>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



### 5. Plot prediccion

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_xgb, "PP")

In [ ]:
plot_prediction(train_data, val_data, test_data, predictions_lgbm, "PP")

## Resumen

In [ ]:
metrics

,Variable,XGBoost MSE,LightGBM MSE
0,WAPP,79.087443,73.531806
1,TP,1468.957694,1320.910009
2,TAD,54.177869,54.025126
3,PP,36.869006,31.712295
